# MGIMO intensive

## Download the data

### 1. Hints

Use the following prompt to develop Python scripts for Jupyter notebooks. Here is a [helpful source](https://agents.md/):

```
## Базовые параметры
- Роль: Junior Python Developer (Data Analyst)
- Специализация: data analysis, data visualization, data collection
- Уровень: начинающий
- Температура: 0 (максимальная точность и предсказуемость)

## Контекст выполнения
Разработка кода для загрузки данных с сайта для последующего анализа.

## Входные данные
- URL ссылка на файл для загрузки `https://storage.yandexcloud.net/tochno-st-catalog/Rosstat/data_bdmo_118_v20250918/by_section/data_section35_112_v20250918_section_file.zip`
- Формат файла: `zip`
- Загруженный файл требуется распаковать и найти файлы данных в архиве

## Технические ограничения
- Использовать стандартные библиотеки Python
- Код запускается в интерактивном ноутбуке Jupyter

## Требования к реализации

### Функциональные требования
1. Чтение файла в ту же директорию, где находится ноутбук
2. Проверка наличия файлов данных в архиве и их размера
3. Гибкая загрузка: возможность менять ссылку для загрузки других файлов

### Технические требования
- Архитектура: упрощенная, для использования в интерактивных ноутбуках
- Стиль кода: PEP 8, black (длина строки 79)
- Документация: Docstrings в стиле Google
- Безопасность: Никаких захардкоженных credentials
```

### 2. Libraries

In [ ]:
import os
import zipfile
from pathlib import Path
from urllib.request import urlretrieve
from urllib.error import URLError

### 3. Download functions

In [ ]:
def download_zip_file(url: str, download_dir: str = ".") -> str:
    """Загружает ZIP-файл по указанной ссылке.

    Args:
        url: URL ссылка на ZIP-файл для загрузки.
        download_dir: Директория для сохранения файла. По умолчанию текущая.

    Returns:
        Путь к загруженному ZIP-файлу.

    Raises:
        URLError: Если не удалось загрузить файл по указанному URL.
    """
    # Создаем директорию, если она не существует
    Path(download_dir).mkdir(parents=True, exist_ok=True)

    # Извлекаем имя файла из URL
    filename = url.split("/")[-1]
    filepath = os.path.join(download_dir, filename)

    print(f"Загрузка файла: {filename}")
    print(f"Источник: {url}")

    try:
        # Загружаем файл
        local_filename, headers = urlretrieve(url, filepath)
        file_size = os.path.getsize(local_filename)
        print(f"Файл успешно загружен: {local_filename}")
        print(f"Размер файла: {file_size:,} байт")
        return local_filename
    except URLError as e:
        print(f"Ошибка загрузки: {e}")
        raise


def explore_zip_archive(zip_path: str) -> list:
    """Исследует содержимое ZIP-архива и проверяет файлы данных.

    Args:
        zip_path: Путь к ZIP-файлу.

    Returns:
        Список информации о файлах в архиве.

    Raises:
        FileNotFoundError: Если ZIP-файл не найден.
        zipfile.BadZipFile: Если файл не является корректным ZIP-архивом.
    """
    if not os.path.exists(zip_path):
        raise FileNotFoundError(f"Файл не найден: {zip_path}")

    print(f"\nАнализ архива: {zip_path}")

    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        file_list = zip_ref.infolist()

        print(f"Всего файлов в архиве: {len(file_list)}")
        print("-" * 60)
        print(f"{'Имя файла':<40} {'Размер (байт)':>15} {'Сжат':>10}")
        print("-" * 60)

        data_files = []
        for file_info in file_list:
            # Проверяем, что это файл (не директория)
            if not file_info.filename.endswith("/"):
                data_files.append(file_info)
                print(
                    f"{file_info.filename:<40} "
                    f"{file_info.file_size:>15,} "
                    f"{file_info.compress_size:>10,}"
                )

        print("-" * 60)
        print(f"Файлов данных найдено: {len(data_files)}")

        # Проверка на пустые файлы
        empty_files = [f for f in data_files if f.file_size == 0]
        if empty_files:
            print("\nВнимание: Найдены пустые файлы:")
            for f in empty_files:
                print(f"  - {f.filename}")

        return data_files


def extract_zip_archive(zip_path: str, extract_dir: str = "extracted_data") -> str:
    """Распаковывает ZIP-архив в указанную директорию.

    Args:
        zip_path: Путь к ZIP-файлу.
        extract_dir: Директория для распаковки.

    Returns:
        Путь к директории с распакованными файлами.
    """
    # Создаем имя директории на основе имени ZIP-файла, если не указано иное
    if extract_dir == "extracted_data":
        zip_name = os.path.splitext(os.path.basename(zip_path))[0]
        extract_dir = os.path.join(".", zip_name)

    print(f"\nРаспаковка в директорию: {extract_dir}")

    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(extract_dir)
        print(f"Архив успешно распакован")

    return extract_dir


def load_and_extract_data(
    url: str, download_dir: str = ".", auto_extract: bool = True
) -> dict:
    """Основная функция для загрузки и распаковки данных.

    Args:
        url: URL для загрузки данных.
        download_dir: Директория для сохранения ZIP-файла.
        auto_extract: Автоматически распаковать архив после загрузки.

    Returns:
        Словарь с информацией о загруженных данных:
        {
            'zip_path': путь к ZIP-файлу,
            'extract_dir': путь к распакованным данным (если auto_extract=True),
            'files': список файлов в архиве
        }
    """
    print("=" * 60)
    print("ЗАГРУЗКА ДАННЫХ")
    print("=" * 60)

    # Загружаем файл
    zip_path = download_zip_file(url, download_dir)

    # Исследуем содержимое архива
    file_list = explore_zip_archive(zip_path)

    result = {"zip_path": zip_path, "files": file_list}

    # Распаковываем, если требуется
    if auto_extract:
        extract_dir = extract_zip_archive(zip_path)
        result["extract_dir"] = extract_dir

        # Показываем содержимое распакованной директории
        print("\nСодержимое распакованной директории:")
        extracted_files = list(Path(extract_dir).glob("*"))
        for file_path in extracted_files:
            size = os.path.getsize(file_path)
            print(f"  - {file_path.name} ({size:,} байт)")

    print("\n" + "=" * 60)
    print("ЗАГРУЗКА ЗАВЕРШЕНА")
    print("=" * 60)

    return result

### 4. Download data

In [ ]:
# URL с данными Росстата
DATA_URL = "https://storage.yandexcloud.net/tochno-st-catalog/Rosstat/data_bdmo_118_v20250918/by_section/data_section35_112_v20250918_section_file.zip"

In [ ]:
# Загружаем и распаковываем данные
result = load_and_extract_data(DATA_URL)

# Доступ к загруженным данным
print(f"\nZIP-файл сохранен: {result['zip_path']}")
print(f"Данные распакованы в: {result['extract_dir']}")

In [ ]:
# Получаем список CSV файлов (если они есть)
data_dir = Path(result["extract_dir"])
csv_files = list(data_dir.glob("*.csv"))
parquet_files = list(data_dir.glob("*.parquet"))
if csv_files:
    print(f"\nНайдено CSV-файлов: {len(csv_files)}")
    for csv_file in csv_files:
        print(f"  - {csv_file.name}")
if parquet_files:
    print(f"\nНайдено PARQUET-файлов: {len(parquet_files)}")
    for parquet_file in parquet_files:
        print(f"  - {parquet_file.name}")

### 5. Download many files

In [ ]:
DATA_URLS = [
    "https://storage.yandexcloud.net/tochno-st-catalog/Rosstat/data_bdmo_118_v20250918/by_section/data_section37_112_v20250918_section_file.zip",
    "https://storage.yandexcloud.net/tochno-st-catalog/Rosstat/data_bdmo_118_v20250918/by_section/data_section39_112_v20250918_section_file.zip",
]

In [ ]:
for data_url in DATA_URLS:
    # Загружаем и распаковываем данные
    result = load_and_extract_data(data_url)
    
    # Доступ к загруженным данным
    print(f"\nZIP-файл сохранен: {result['zip_path']}")
    print(f"Данные распакованы в: {result['extract_dir']}")